In [1]:
%pwd

'/home/howard/ml_tensor'

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
import tensorflow as tf

In [5]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [8]:
reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [9]:
type(housing.data)

numpy.ndarray

In [10]:
m

20640

In [11]:
n

8

In [12]:
type(housing.target)

numpy.ndarray

In [13]:
housing.target.shape

(20640,)

In [14]:
housing.target[:5]

array([4.526, 3.585, 3.521, 3.413, 3.422])

In [15]:
housing.data[:5]

array([[ 8.32520000e+00,  4.10000000e+01,  6.98412698e+00,
         1.02380952e+00,  3.22000000e+02,  2.55555556e+00,
         3.78800000e+01, -1.22230000e+02],
       [ 8.30140000e+00,  2.10000000e+01,  6.23813708e+00,
         9.71880492e-01,  2.40100000e+03,  2.10984183e+00,
         3.78600000e+01, -1.22220000e+02],
       [ 7.25740000e+00,  5.20000000e+01,  8.28813559e+00,
         1.07344633e+00,  4.96000000e+02,  2.80225989e+00,
         3.78500000e+01, -1.22240000e+02],
       [ 5.64310000e+00,  5.20000000e+01,  5.81735160e+00,
         1.07305936e+00,  5.58000000e+02,  2.54794521e+00,
         3.78500000e+01, -1.22250000e+02],
       [ 3.84620000e+00,  5.20000000e+01,  6.28185328e+00,
         1.08108108e+00,  5.65000000e+02,  2.18146718e+00,
         3.78500000e+01, -1.22250000e+02]])

In [17]:
type(housing_data_plus_bias)

numpy.ndarray

In [18]:
housing_data_plus_bias[:5]

array([[ 1.00000000e+00,  8.32520000e+00,  4.10000000e+01,
         6.98412698e+00,  1.02380952e+00,  3.22000000e+02,
         2.55555556e+00,  3.78800000e+01, -1.22230000e+02],
       [ 1.00000000e+00,  8.30140000e+00,  2.10000000e+01,
         6.23813708e+00,  9.71880492e-01,  2.40100000e+03,
         2.10984183e+00,  3.78600000e+01, -1.22220000e+02],
       [ 1.00000000e+00,  7.25740000e+00,  5.20000000e+01,
         8.28813559e+00,  1.07344633e+00,  4.96000000e+02,
         2.80225989e+00,  3.78500000e+01, -1.22240000e+02],
       [ 1.00000000e+00,  5.64310000e+00,  5.20000000e+01,
         5.81735160e+00,  1.07305936e+00,  5.58000000e+02,
         2.54794521e+00,  3.78500000e+01, -1.22250000e+02],
       [ 1.00000000e+00,  3.84620000e+00,  5.20000000e+01,
         6.28185328e+00,  1.08108108e+00,  5.65000000e+02,
         2.18146718e+00,  3.78500000e+01, -1.22250000e+02]])

In [22]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [23]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [24]:
theta_value

array([[-3.6959320e+01],
       [ 4.3698898e-01],
       [ 9.4245886e-03],
       [-1.0791138e-01],
       [ 6.4842808e-01],
       [-3.9986235e-06],
       [-3.7866351e-03],
       [-4.2142656e-01],
       [-4.3467718e-01]], dtype=float32)

In [25]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)


[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [20]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [42]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


In [43]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)


In [44]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
Best theta:
[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178365]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


## Mini-batch Gradient Descent

In [26]:
n_epochs = 1000
learning_rate = 0.01

In [27]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [28]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [29]:
n_epochs = 10

In [30]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [31]:
n_batches 

207

In [32]:
m

20640

In [33]:
n

8

In [40]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    print('seed:',epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    #print((X_batch))
    #print((y_batch))
    #print(indices)
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        print("epoch:",epoch)
        for batch_index in range(n_batches):
            print("batch_index:",batch_index)
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

epoch: 0
batch_index: 0
seed: 0
batch_index: 1
seed: 1
batch_index: 2
seed: 2
batch_index: 3
seed: 3
batch_index: 4
seed: 4
batch_index: 5
seed: 5
batch_index: 6
seed: 6
batch_index: 7
seed: 7
batch_index: 8
seed: 8
batch_index: 9
seed: 9
batch_index: 10
seed: 10
batch_index: 11
seed: 11
batch_index: 12
seed: 12
batch_index: 13
seed: 13
batch_index: 14
seed: 14
batch_index: 15
seed: 15
batch_index: 16
seed: 16
batch_index: 17
seed: 17
batch_index: 18
seed: 18
batch_index: 19
seed: 19
batch_index: 20
seed: 20
batch_index: 21
seed: 21
batch_index: 22
seed: 22
batch_index: 23
seed: 23
batch_index: 24
seed: 24
batch_index: 25
seed: 25
batch_index: 26
seed: 26
batch_index: 27
seed: 27
batch_index: 28
seed: 28
batch_index: 29
seed: 29
batch_index: 30
seed: 30
batch_index: 31
seed: 31
batch_index: 32
seed: 32
batch_index: 33
seed: 33
batch_index: 34
seed: 34
batch_index: 35
seed: 35
batch_index: 36
seed: 36
batch_index: 37
seed: 37
batch_index: 38
seed: 38
batch_index: 39
seed: 39
batch_index

batch_index: 170
seed: 377
batch_index: 171
seed: 378
batch_index: 172
seed: 379
batch_index: 173
seed: 380
batch_index: 174
seed: 381
batch_index: 175
seed: 382
batch_index: 176
seed: 383
batch_index: 177
seed: 384
batch_index: 178
seed: 385
batch_index: 179
seed: 386
batch_index: 180
seed: 387
batch_index: 181
seed: 388
batch_index: 182
seed: 389
batch_index: 183
seed: 390
batch_index: 184
seed: 391
batch_index: 185
seed: 392
batch_index: 186
seed: 393
batch_index: 187
seed: 394
batch_index: 188
seed: 395
batch_index: 189
seed: 396
batch_index: 190
seed: 397
batch_index: 191
seed: 398
batch_index: 192
seed: 399
batch_index: 193
seed: 400
batch_index: 194
seed: 401
batch_index: 195
seed: 402
batch_index: 196
seed: 403
batch_index: 197
seed: 404
batch_index: 198
seed: 405
batch_index: 199
seed: 406
batch_index: 200
seed: 407
batch_index: 201
seed: 408
batch_index: 202
seed: 409
batch_index: 203
seed: 410
batch_index: 204
seed: 411
batch_index: 205
seed: 412
batch_index: 206
seed: 413
e

batch_index: 140
seed: 761
batch_index: 141
seed: 762
batch_index: 142
seed: 763
batch_index: 143
seed: 764
batch_index: 144
seed: 765
batch_index: 145
seed: 766
batch_index: 146
seed: 767
batch_index: 147
seed: 768
batch_index: 148
seed: 769
batch_index: 149
seed: 770
batch_index: 150
seed: 771
batch_index: 151
seed: 772
batch_index: 152
seed: 773
batch_index: 153
seed: 774
batch_index: 154
seed: 775
batch_index: 155
seed: 776
batch_index: 156
seed: 777
batch_index: 157
seed: 778
batch_index: 158
seed: 779
batch_index: 159
seed: 780
batch_index: 160
seed: 781
batch_index: 161
seed: 782
batch_index: 162
seed: 783
batch_index: 163
seed: 784
batch_index: 164
seed: 785
batch_index: 165
seed: 786
batch_index: 166
seed: 787
batch_index: 167
seed: 788
batch_index: 168
seed: 789
batch_index: 169
seed: 790
batch_index: 170
seed: 791
batch_index: 171
seed: 792
batch_index: 172
seed: 793
batch_index: 173
seed: 794
batch_index: 174
seed: 795
batch_index: 175
seed: 796
batch_index: 176
seed: 797
b

batch_index: 142
seed: 1177
batch_index: 143
seed: 1178
batch_index: 144
seed: 1179
batch_index: 145
seed: 1180
batch_index: 146
seed: 1181
batch_index: 147
seed: 1182
batch_index: 148
seed: 1183
batch_index: 149
seed: 1184
batch_index: 150
seed: 1185
batch_index: 151
seed: 1186
batch_index: 152
seed: 1187
batch_index: 153
seed: 1188
batch_index: 154
seed: 1189
batch_index: 155
seed: 1190
batch_index: 156
seed: 1191
batch_index: 157
seed: 1192
batch_index: 158
seed: 1193
batch_index: 159
seed: 1194
batch_index: 160
seed: 1195
batch_index: 161
seed: 1196
batch_index: 162
seed: 1197
batch_index: 163
seed: 1198
batch_index: 164
seed: 1199
batch_index: 165
seed: 1200
batch_index: 166
seed: 1201
batch_index: 167
seed: 1202
batch_index: 168
seed: 1203
batch_index: 169
seed: 1204
batch_index: 170
seed: 1205
batch_index: 171
seed: 1206
batch_index: 172
seed: 1207
batch_index: 173
seed: 1208
batch_index: 174
seed: 1209
batch_index: 175
seed: 1210
batch_index: 176
seed: 1211
batch_index: 177
see

batch_index: 81
seed: 1530
batch_index: 82
seed: 1531
batch_index: 83
seed: 1532
batch_index: 84
seed: 1533
batch_index: 85
seed: 1534
batch_index: 86
seed: 1535
batch_index: 87
seed: 1536
batch_index: 88
seed: 1537
batch_index: 89
seed: 1538
batch_index: 90
seed: 1539
batch_index: 91
seed: 1540
batch_index: 92
seed: 1541
batch_index: 93
seed: 1542
batch_index: 94
seed: 1543
batch_index: 95
seed: 1544
batch_index: 96
seed: 1545
batch_index: 97
seed: 1546
batch_index: 98
seed: 1547
batch_index: 99
seed: 1548
batch_index: 100
seed: 1549
batch_index: 101
seed: 1550
batch_index: 102
seed: 1551
batch_index: 103
seed: 1552
batch_index: 104
seed: 1553
batch_index: 105
seed: 1554
batch_index: 106
seed: 1555
batch_index: 107
seed: 1556
batch_index: 108
seed: 1557
batch_index: 109
seed: 1558
batch_index: 110
seed: 1559
batch_index: 111
seed: 1560
batch_index: 112
seed: 1561
batch_index: 113
seed: 1562
batch_index: 114
seed: 1563
batch_index: 115
seed: 1564
batch_index: 116
seed: 1565
batch_index

batch_index: 185
seed: 1841
batch_index: 186
seed: 1842
batch_index: 187
seed: 1843
batch_index: 188
seed: 1844
batch_index: 189
seed: 1845
batch_index: 190
seed: 1846
batch_index: 191
seed: 1847
batch_index: 192
seed: 1848
batch_index: 193
seed: 1849
batch_index: 194
seed: 1850
batch_index: 195
seed: 1851
batch_index: 196
seed: 1852
batch_index: 197
seed: 1853
batch_index: 198
seed: 1854
batch_index: 199
seed: 1855
batch_index: 200
seed: 1856
batch_index: 201
seed: 1857
batch_index: 202
seed: 1858
batch_index: 203
seed: 1859
batch_index: 204
seed: 1860
batch_index: 205
seed: 1861
batch_index: 206
seed: 1862
epoch: 9
batch_index: 0
seed: 1863
batch_index: 1
seed: 1864
batch_index: 2
seed: 1865
batch_index: 3
seed: 1866
batch_index: 4
seed: 1867
batch_index: 5
seed: 1868
batch_index: 6
seed: 1869
batch_index: 7
seed: 1870
batch_index: 8
seed: 1871
batch_index: 9
seed: 1872
batch_index: 10
seed: 1873
batch_index: 11
seed: 1874
batch_index: 12
seed: 1875
batch_index: 13
seed: 1876
batch_i

In [41]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255152],
       [-0.31211877],
       [ 0.38510376],
       [ 0.00434168],
       [-0.0123295 ],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

In [ ]:
Best theta:
[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178365]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]